# NAVIGATION PROJECT REPORT

---

_1/17/2021_


This is the first project in the Udacity Deep Reinforcement Learning nano-degree program.  The goal is to build a DQN agent that learns to move around a flat, rectangular environment, collecting the yellow bananas it sees while avoiding the blue bananas.  Rather than using raw pixel inputs from the visual display, as is customary for agents playing the Atari games, this agent will receive preprocessed state data that represents ray-based observations of the environment in front of it, as well as its own position and velocities.

The top level code for this project is in the 'Navigation.ipynb' Jupyter notebook.  That notebook further calls on two classes in separate Python files in the same directory.  See [this file](navigation_readme.md) for more info on the project itself.

## Algorithm

This project used code from an earlier exercise in the Udacity DRL program as a starting baseline, namely the lunar lander exercise, which employed Q-learning.  Here, many enhancements have been added to that code, the largest of which is a three-layer neural network to approximate the Q value function and replace the Q-table, thus making it a Deep Q-learning model.  

### Neural Network

The neural network employed here uses three fully-connected layers:
- Layer 1 has 370 nodes and leaky-RELU activation
- Layer 2 has 296 nodes and leaky-RELU activation
- Layer 3 has 4 nodes and no activation; it outputs the Q values of the four possible actions that the agent can take.  I had experimented with variations on this structure, including varying the sizes of the layers, the slope of the leaky-RELU activation, and the use of dropouts.  In the end, dropout was not used at all.

### Fixed-Q Targets

The agent relies upon Fixed-Q targets, where the target Q values are generated by a separate NN (named _qnetwork_target_ ) from the actual solution network (which is named _qnetwork_local_ ).  With this critical enhancement the target values for finding the Q adjustments necessary for learning are held constant for a few sampling iterations (in this case the hyperparameter ( _UPDATE_EVERY_ ) is set at 4 time steps).  Once that number of time steps is collected, the learning process is invoked, and the targets are then incrementally updated to get closer to the local model's Q values.  They values are not directly copied into the target network, rather a _TAU_ hyperparameter dictates that they are moved only slightly in that direction for each learning step.  By moving the target gradually, the learning process is better stabilized.

### Experience Replay

Another enhancement that was added to the algorithm was Experience Replay.  There were two iterations of this enhancement.  In the first iteration, it used instructor-provided code for the _ReplayBuffer_ class that appears in the code notebook.  This simple buffer provides a uniform probability replay functionality.  Because of the sequential nature of a temporal control problem, the agent's experience at a given time step has some correlation to the experience from the previous time step.  Proper use of the Bellman equation for NN learning assumes no correlation between training data points.  In order to approach this lack of correlation, the agent's experiences (time step results) are not fed into the NN training algorithm as soon as they appear.  Rather, then are stored in the experience replay buffer, which holds a large number of historical experiences.  Then at each learning step the replay buffer is randomly sampled for a batch of experience data points that are normally far removed from each other in the time domain.  This random sampling provides a very low probability of correlation between data points within a given batch of training data.

### Prioritized Experience Replay (PER)

After the solution was running satisfactorily with the initial experience replay capability, the next major enhnacement was to add PER,which is described [in this paper](https://arxiv.org/pdf/1511.05952.pdf).  The idea here is that some experiences are more valuable "learning moments" than others, and therefore they should be emphasized more in the training process.  One measure of an experience's importance is the loss, or error, calculated between the Q value that it generated and the expected, or target, Q value for that state-action pair.  If we can skew the random sampling of the replay buffer to give preference to those experiences with a higher error, then our model should learn to be more responsive in these more non-linear situations, and therefore generalize better or faster.

Implementing PER is a somewhat complex endeavor.  The existing _ReplayBoffer_ class had to be abandoned for an entirely different approach (although the code has been left in the solution notebook for reference).  A new data structure is needed in order to store and retrieve the experiences based on a variable importance (hereinafter called priority).  The SumTree structure is ideally suited to this application.  It is a tree structure in which each node's value is the sum of the values of its (zero to two) children.  With each leaf node holding one experience point, and its tree value being the priority of that experience point, the tree can store, update and retrieve nodes in O(logN) time.  A good explanation of how the structure works [can be found here](https://adventuresinmachinelearning.com/sumtree-introduction-python/).  In my solution the SumTree code is stored, not in the solution notebook, but in a separate flat file in the same directory, as it was built on another author's base code.

Finally, another class is needed to wrap the SumTree structure and manage its use for this particular application.  The PrioritizedReplay class, like the SumTree class, is stored in a separate Python file in this solution directory.  It, too, is built upon base code reused from another author.  This class implements the additional mathematical complexity necessary for PER.  First, we have a hyperparameter, _alpha_ in [0, 1], that can dial in how much we want the algorithm to sample based on priority.  If alpha = 1 then it distributes the probability space proportional to each node's priority.  If alpha = 0 then the probability distribution is uniform, completely ignoring the node priorities.  For alpha > 0, we are no longer using a uniform random distribution, thus a bias would be introduced that risks destabilizing the training algorithm.  To compensate for that bias, an Importance Sampling (IS) weight is defined.  This weight is then multiplied by the Q error in the learning process.  We can control this weight by another new hyperparemeter, _beta_ in [0, 1].  The larger beta, the more it compensates for a non-uniform probability distribution.  Both the alpha and beta hyperparameters are recommended to start at somewhat small values the be annealed toward 1 as the learning process comes to completion.

### Double DQN

The final enhancement used in this project is Double DQN.  This technique is closely related to the fixed-Q targets, and modifies how the targets are calculated.  This is because the targets have a tendency to be over-estimated without such a correction.  Rather than selecting the largest action value for any given state, we want to use two separate NNs in this process; one to select the best action then the other one to evaluate that selection.  These two NNs need to have separate sets of weights, so that they have different perspectives on the choice of action.  If they both agree, then the chosen action gets updates much as would be done before.  If they disagree, then a much smaller update is applied to the Q value in question.  Since we are already using the Fixed-Q Learning enhancement, which uses two separate NNs with different weights, we just use these same NNs for the Double DQN implementation.

## Results

As this class has taught me so far, DQN solutions are finicky and difficult to train.  Small adjustments to the set of hyperparameters can be the difference between a solid solution and one that doesn't coverge at all.  Therefore, training is heavily dependent upon trying various combinations of hyperparameters and making judgment calls about how they might interact.

For this project I first used just Fixed-Q learning and simple Experience Replay.  I got a solution that worked well, converging to a solution (100-episode average score >= +13) in 598 episodes.  

I then added Double DQN and was able to achieve similar results, averaging 499 episodes (over 3 runs) to solve the problem.

Finally, I added PER.  This enhancement was disappointing in that I was unable to get any convergence unless I set both alpha and beta to zero (turning off the prioritization).  I was able to get a solution if I allowed them to start annealing (alpha grew from 0 to 0.05 and beta grew from 0 to 0.18).  With this setting it converged in around 680 episodes.  If I turned off the annealing (keeping both alpha and beta fixed at 0), then the solution converged in arounc 650 episodes.  I did many runs with various combinations of initial alpha, initial beta, annealing rates, and even played with the _tau_ and _learning rate_ hyperparams, but was never able to get a converged solution with any significant prioritizing at all.  I also spent lots of time running with print statements enabled to ensure that the code is working correctly.  My conclusion is that this particular game is not an environment where PER provides an advantage.  I found a mentor post that suggested the same thing.

The below plot show the results with the full set of code described above, with PER alpha and beta fixed at 0.
![plot](results_plot.png)

## Future Considerations

I was hoping to add the Dualing DQN enhancement as well, but will save that exploration for sometime after completing the course, due to time constraints.  It promises significant improvements for what this agent can do.  Beyond that, the Rainbow Algorithm described in class looks like it could provide even further improvements.

I also want to apply the code I have here to other environments.  I am particularly interested to understand through experience where PER can be more of an advantage.  I suspect that it will start to shine where more of a continuous control output is required, and where the environment has more "smooth" responses to many commands (the banana game is a chaotic environment where large + or - rewards are coming in frequently, and there is nothing in between).